# Classification 종합실습

 ## 신용대출 심사

* 고객사는 ## 은행입니다. 신용평가 업무를 인공지능으로 전환하고자 여러분에게 모델링을 의뢰하였습니다.
* 대출업무는
    * 은행 창구에서 신청을 받고
    * 본사의 심사부서에서는 신용평가를 통해 대출 신청에 대한 승인 여부를 결정해 왔습니다.

* 현장의 요구
    * 경쟁사의 공격적인 대출상품 판매로, 본사에서는 자사 은행의 대출 실적이 줄어들고 있는 것에 부담을 느끼고 있습니다.
    * 그런데, 자사 은행에서는 신용평가 결과의 정확성에 의문을 품고 있으며, 신용평가 기준을 완화하여 가급적 대출승인 범위를 더 확대해 주기를 요구합니다. 

* 신용평가 업무를 인공지능으로 전환
    * 현장의 요구를 감안하여, 과거 사람이 하던 평가방식을 개선하고자 인공지능에 의한 예측 모델을 만들고, 정확도를 높이고자 합니다.
    * 최적의 모델을 생성해 봅시다.


## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

In [2]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/credit_all.csv'
data = pd.read_csv(path)
data.loc[data['Payment'] == 4, 'Payment'] = 3
data.head()

,Creditability,AccountBalance,Duration,Payment,Purpose,CreditAmount,Employment,SexMarital,CurrentAddress,MostValuableAsset,Age,Apartment,CreditCount,Occupation,Dependents,Telephone,ForeignWorker
0,1,3,24,2,0,1249,2,4,2,1,28,2,1,3,1,1,1
1,1,2,9,2,0,276,3,4,4,1,22,1,1,2,1,1,1
2,1,1,18,3,2,1049,2,2,4,2,21,1,1,3,1,1,1
3,1,1,24,3,1,6419,5,2,4,4,44,3,2,4,2,2,1
4,1,3,12,2,2,1424,5,2,4,1,55,2,1,4,1,2,1


|	칼럼명	|	설명	|	 	|	값 의미	|
|	-----	|	-----	|	-----	|	-----	|
|	Creditability	|	Creditability(Target)	|	향후 신용도	|	0 : Bad, 1 : Good	|
|	AccountBalance	|	Account Balance	|	은행잔고	|	1: No account, 2 : None (No balance), 3 : Some Balance	|
|	CreditDuration	|	Duration of Credit (month)	|	신청한 대출기간(월)	|	숫자	|
|	Payment	|	Payment Status of Previous Credit	|	과거 대출 납입 상태	|	0 : 연체, 1 : 기타신용, 2 : 완납, 3 : 정상 대출상환 중 |
|	Purpose	|	Purpose	|	신청한 대출목적	|	1 : New Car , 2 : Used Car , 3 : Furniture , 4 : TV , 5 : Appliances , 6 : Repair , 8 : Vacation , 9 :Retraining , 10 : Business , 0 : Other	|
|	CreditAmount	|	Credit Amount($)	|	신청한 대출금액	|		|
|	Employment	|	Length of current employment(Month)	|	현 직업 근무 기간	|	1: Unemployed,  2: <1 Year,  3: [1, 4),  4: [4, 7),  5: Above 7	|
|	SexMarital	|	Sex & Marital Status	|	성별 & 결혼상태	|	1: Male, Divorced, 2: Male, Single , 3: Male, Married/Widowed , 4: Female	|
|	CurrentAddress	|	Duration in Current address	|	현 거주지 거주기간	|	1: <1 Year , 2: [1, 4) , 3: [4, 7) , 4: Above 7	|
|	MostValuable	|	Most valuable available asset	|	가장 가치있는 자산	|	1: None , 2: Car , 3: Life Insurance , 4: Real Estate	|
|	Age	|	Age (years)	|	나이	|		|
|	AppartmentType	|	Type of apartment	|	주거환경	|	1: free apartment, 2: Rented, 3: Owned	|
|	NoCredits	|	No of Credits at this Bank	|	현재 총 대출 건수	|	1 : one, 2 : 2 ~ 3, 3 : 4 ~ 5, 4 : 6 ~	|
|	Occupation	|	Occupation	|	직업	|	1: Unemployed, unskilled, 2: Unskilled Permanent Resident, 3: Skilled, 4: Executive	|
|	Telephone	|	Telephone	|	전화기 소유 여부	|	2: Yes , 1: No	|
|	ForeignWorker	|	Foreign Worker	|	외국인 근로자 여부	|	2: Yes , 1: No	|


## 2.데이터 준비

### (1) 데이터 정리

### (2) 데이터분할1 : x, y 나누기

In [3]:
target = 'Creditability'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [4]:
dummy_vars = ['Employment', 'CurrentAddress', 'CreditCount', 'Dependents', 'Telephone', 'AccountBalance', 'Payment', 'Purpose', 'SexMarital', 'MostValuableAsset', 'Apartment','Occupation','ForeignWorker']
x = pd.get_dummies(x, columns = dummy_vars, drop_first = True)
x.head()

,Duration,CreditAmount,Age,Employment_2,Employment_3,Employment_4,Employment_5,CurrentAddress_2,CurrentAddress_3,CurrentAddress_4,...,SexMarital_4,MostValuableAsset_2,MostValuableAsset_3,MostValuableAsset_4,Apartment_2,Apartment_3,Occupation_2,Occupation_3,Occupation_4,ForeignWorker_2
0,24,1249,28,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,1,0,0
1,9,276,22,0,1,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,18,1049,21,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
3,24,6419,44,0,0,0,1,0,0,1,...,0,0,0,1,0,1,0,0,1,0
4,12,1424,55,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0


### (5) 데이터분할2 : train : validation 나누기

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 2022)

### (6) Scaling

In [6]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.모델링

* 사용 알고리즘 : LogisticRegression, DecisionTreeClassifier, KNeighborsClassifier, SVC
* 3가지의 알고리즘을 선정하고 성능을 튜닝해 봅시다.

### (1) 로지스틱 회귀

* 함수 생성

In [7]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

* 전진선택법 수행

In [8]:
vars, result = forward_stepwise_logistic(x_train,y_train)

Optimization terminated successfully.
         Current function value: 0.655913
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.667719
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.591570
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.692746
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.660513
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.664357
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.652765
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.667777
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.682336
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.651675
  

Optimization terminated successfully.
         Current function value: 0.535686
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.536069
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.536119
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.533815
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.532943
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.535570
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.535591
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.535546
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.535167
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.533276
  

Optimization terminated successfully.
         Current function value: 0.510098
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.510167
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.510118
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.509047
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.507436
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.510180
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.510193
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.507939
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.509887
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.509711
  

Optimization terminated successfully.
         Current function value: 0.490674
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.486088
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.485475
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.485117
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.486657
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.484734
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.486397
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.486633
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.484489
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.486655
  

Optimization terminated successfully.
         Current function value: 0.475497
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.476518
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.474061
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.476258
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.476658
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.473885
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.476665
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.476659
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.476297
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.475300
  

Optimization terminated successfully.
         Current function value: 0.468654
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468667
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.467365
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468597
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.467660
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468912
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468578
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468405
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468374
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.468366
  

* 선택된 변수

In [9]:
vars

['AccountBalance_3',
 'Payment_3',
 'Purpose_1',
 'CreditAmount',
 'Payment_2',
 'Duration',
 'Employment_2',
 'AccountBalance_2',
 'ForeignWorker_2',
 'Purpose_3',
 'SexMarital_3',
 'CreditCount_2',
 'CurrentAddress_2',
 'MostValuableAsset_4',
 'Telephone_2',
 'Occupation_4']

* 전체 변수로 모델링

In [10]:
list(x_train)

['Duration',
 'CreditAmount',
 'Age',
 'Employment_2',
 'Employment_3',
 'Employment_4',
 'Employment_5',
 'CurrentAddress_2',
 'CurrentAddress_3',
 'CurrentAddress_4',
 'CreditCount_2',
 'CreditCount_3',
 'CreditCount_4',
 'Dependents_2',
 'Telephone_2',
 'AccountBalance_2',
 'AccountBalance_3',
 'Payment_1',
 'Payment_2',
 'Payment_3',
 'Purpose_1',
 'Purpose_2',
 'Purpose_3',
 'Purpose_4',
 'Purpose_5',
 'Purpose_6',
 'Purpose_8',
 'Purpose_9',
 'Purpose_10',
 'SexMarital_2',
 'SexMarital_3',
 'SexMarital_4',
 'MostValuableAsset_2',
 'MostValuableAsset_3',
 'MostValuableAsset_4',
 'Apartment_2',
 'Apartment_3',
 'Occupation_2',
 'Occupation_3',
 'Occupation_4',
 'ForeignWorker_2']

* 전진선택법 변수로 모델링

In [11]:
result

,step,feature,aic
0,1.0,[AccountBalance_3],766.998784
1,1.0,[Age],830.197635
2,1.0,[Apartment_2],844.952933
3,1.0,[Payment_3],861.716743
4,1.0,[SexMarital_3],866.666531
...,...,...,...
531,16.0,"[AccountBalance_3, Payment_3, Purpose_1, Credi...",686.277690
532,16.0,"[AccountBalance_3, Payment_3, Purpose_1, Credi...",686.292828
533,16.0,"[AccountBalance_3, Payment_3, Purpose_1, Credi...",686.295740
534,16.0,"[AccountBalance_3, Payment_3, Purpose_1, Credi...",686.306342


In [13]:
lr = LogisticRegression()
lr.fit(x_train[vars],y_train)
p1 = lr.predict(x_val[vars])

print(accuracy_score(y_val,p1))
print(classification_report(y_val,p1))

0.7333333333333333
              precision    recall  f1-score   support

           0       0.62      0.46      0.53        97
           1       0.77      0.86      0.81       203

    accuracy                           0.73       300
   macro avg       0.69      0.66      0.67       300
weighted avg       0.72      0.73      0.72       300



### (2) 의사결정나무

In [14]:
DT = DecisionTreeClassifier()

In [15]:
DT.fit(x_train,y_train)

DecisionTreeClassifier()

In [16]:
p2 = DT.predict(x_val)
print(accuracy_score(y_val,p2))
print(classification_report(y_val,p2))

0.6433333333333333
              precision    recall  f1-score   support

           0       0.45      0.51      0.48        97
           1       0.75      0.71      0.73       203

    accuracy                           0.64       300
   macro avg       0.60      0.61      0.60       300
weighted avg       0.65      0.64      0.65       300



In [24]:
params = {'max_depth': range(1,10),'min_samples_leaf' : range(10,101,10)}

In [25]:
rm2 = DecisionTreeClassifier()
model_rs2 = RandomizedSearchCV(rm2,params,cv=5,n_iter=5)

In [26]:
model_rs2.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=5,
                   param_distributions={'max_depth': range(1, 10),
                                        'min_samples_leaf': range(10, 101, 10)})

In [28]:
model_rs2.cv_results_

{'mean_fit_time': array([0.00579295, 0.00369725, 0.0038857 , 0.0034513 , 0.00356035]),
 'std_fit_time': array([0.00095539, 0.00034437, 0.00070447, 0.00087646, 0.00040316]),
 'mean_score_time': array([0.00287156, 0.00200353, 0.0018064 , 0.00181189, 0.00208549]),
 'std_score_time': array([0.00043878, 0.00067183, 0.00041825, 0.00075182, 0.00090988]),
 'param_min_samples_leaf': masked_array(data=[70, 40, 60, 80, 50],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[2, 1, 6, 3, 3],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 70, 'max_depth': 2},
  {'min_samples_leaf': 40, 'max_depth': 1},
  {'min_samples_leaf': 60, 'max_depth': 6},
  {'min_samples_leaf': 80, 'max_depth': 3},
  {'min_samples_leaf': 50, 'max_depth': 3}],
 'split0_test_score': array([0.75714286, 0.70714286, 0.75714286, 0.75714286, 0.76

In [29]:
model_rs2.best_params_

{'min_samples_leaf': 80, 'max_depth': 3}

In [30]:
model_rs2.best_score_

0.7428571428571429

In [32]:
pred2 = model_rs2.predict(x_val)
print(classification_report(y_val,pred2))

              precision    recall  f1-score   support

           0       0.55      0.46      0.50        97
           1       0.76      0.82      0.79       203

    accuracy                           0.70       300
   macro avg       0.66      0.64      0.65       300
weighted avg       0.69      0.70      0.70       300



### (3) KNN

In [17]:
KN = KNeighborsClassifier()

In [18]:
KN.fit(x_train_s,y_train)

KNeighborsClassifier()

In [19]:
p3 = KN.predict(x_val_s)
print(accuracy_score(y_val,p3))
print(classification_report(y_val,p3))

0.6733333333333333
              precision    recall  f1-score   support

           0       0.49      0.29      0.36        97
           1       0.72      0.86      0.78       203

    accuracy                           0.67       300
   macro avg       0.60      0.57      0.57       300
weighted avg       0.64      0.67      0.65       300



In [34]:
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }

In [35]:
KN2 = KNeighborsClassifier()

KN2_rs = RandomizedSearchCV(KN2,params,cv=5,n_iter=5)

In [36]:
KN2_rs.fit(x_train_s,y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [37]:
KN2_rs.cv_results_

{'mean_fit_time': array([0.00130572, 0.00120821, 0.00116997, 0.00106497, 0.0012804 ]),
 'std_fit_time': array([0.00038211, 0.00039389, 0.000327  , 0.0006406 , 0.00049453]),
 'mean_score_time': array([0.01222858, 0.00913706, 0.01132908, 0.01251554, 0.01203833]),
 'std_score_time': array([0.00203978, 0.00061653, 0.00039523, 0.00078932, 0.0006544 ]),
 'param_n_neighbors': masked_array(data=[11, 33, 13, 44, 28],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['manhattan', 'euclidean', 'manhattan', 'manhattan',
                    'manhattan'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 11, 'metric': 'manhattan'},
  {'n_neighbors': 33, 'metric': 'euclidean'},
  {'n_neighbors': 13, 'metric': 'manhattan'},
  {'n_neighbors': 44, 'metric': 'manhattan'},
  {'n_neighbors': 28, 'metric': 'manhattan'}],
 'split0_

In [39]:
KN2_rs.cv_results_['params']

[{'n_neighbors': 11, 'metric': 'manhattan'},
 {'n_neighbors': 33, 'metric': 'euclidean'},
 {'n_neighbors': 13, 'metric': 'manhattan'},
 {'n_neighbors': 44, 'metric': 'manhattan'},
 {'n_neighbors': 28, 'metric': 'manhattan'}]

In [40]:
KN2_rs.cv_results_['mean_test_score']

array([0.70571429, 0.70857143, 0.70571429, 0.70571429, 0.72      ])

In [41]:
KN2_rs.best_params_

{'n_neighbors': 28, 'metric': 'manhattan'}

In [42]:
KN2_rs.best_score_

0.72

In [43]:
pred3 = KN2_rs.predict(x_val_s)
print(classification_report(y_val,pred3))

              precision    recall  f1-score   support

           0       0.54      0.07      0.13        97
           1       0.69      0.97      0.80       203

    accuracy                           0.68       300
   macro avg       0.61      0.52      0.47       300
weighted avg       0.64      0.68      0.59       300



### (4) SVM

In [20]:
SV = SVC()

In [21]:
SV.fit(x_train_s,y_train)

SVC()

In [22]:
p4 = SV.predict(x_val_s)
print(accuracy_score(y_val,p4))
print(classification_report(y_val,p4))

0.7233333333333334
              precision    recall  f1-score   support

           0       0.63      0.34      0.44        97
           1       0.74      0.91      0.82       203

    accuracy                           0.72       300
   macro avg       0.69      0.62      0.63       300
weighted avg       0.71      0.72      0.70       300



In [63]:
params = {'C': np.linspace(0.1,5,10),'gamma' : np.linspace(0.1,5,10)}

In [64]:
SV2 = SVC()

SV2_rs = RandomizedSearchCV(SV2,params,cv=5,n_iter=5)

In [65]:
SV2_rs.fit(x_train_s,y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=5,
                   param_distributions={'C': array([0.1       , 0.64444444, 1.18888889, 1.73333333, 2.27777778,
       2.82222222, 3.36666667, 3.91111111, 4.45555556, 5.        ]),
                                        'gamma': array([0.1       , 0.64444444, 1.18888889, 1.73333333, 2.27777778,
       2.82222222, 3.36666667, 3.91111111, 4.45555556, 5.        ])})

In [66]:
SV2_rs.cv_results_

{'mean_fit_time': array([0.02374387, 0.06893377, 0.03232732, 0.02888155, 0.03157978]),
 'std_fit_time': array([0.00418796, 0.01827604, 0.00281821, 0.00169468, 0.00065167]),
 'mean_score_time': array([0.01091542, 0.02430458, 0.01494393, 0.01342816, 0.0124023 ]),
 'std_score_time': array([0.00265621, 0.00422083, 0.00411573, 0.00110044, 0.00097672]),
 'param_gamma': masked_array(data=[0.1, 0.6444444444444445, 0.6444444444444445,
                    1.188888888888889, 4.455555555555556],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[0.6444444444444445, 4.455555555555556,
                    3.911111111111112, 1.7333333333333336,
                    3.911111111111112],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'gamma': 0.1, 'C': 0.6444444444444445},
  {'gamma': 0.6444444444444445, 'C': 4.455555555555556},
  {'gamma': 0.64

In [67]:
SV2_rs.cv_results_['params']

[{'gamma': 0.1, 'C': 0.6444444444444445},
 {'gamma': 0.6444444444444445, 'C': 4.455555555555556},
 {'gamma': 0.6444444444444445, 'C': 3.911111111111112},
 {'gamma': 1.188888888888889, 'C': 1.7333333333333336},
 {'gamma': 4.455555555555556, 'C': 3.911111111111112}]

In [68]:
SV2_rs.cv_results_['mean_test_score']

array([0.71142857, 0.70285714, 0.70285714, 0.71      , 0.71      ])

In [69]:
SV2_rs.best_params_

{'gamma': 0.1, 'C': 0.6444444444444445}

In [70]:
SV2_rs.best_score_

0.7114285714285715

In [71]:
pred4 = SV2_rs.predict(x_val_s)
print(classification_report(y_val,pred4))

              precision    recall  f1-score   support

           0       0.71      0.12      0.21        97
           1       0.70      0.98      0.81       203

    accuracy                           0.70       300
   macro avg       0.70      0.55      0.51       300
weighted avg       0.70      0.70      0.62       300

